In [2]:
import win32evtlog
import csv

def fetch_windows_logs(log_types=None):
    if log_types is None:
        log_types = ["Application", "Security", "System", "Setup", "ForwardedEvents"]
    
    for log_type in log_types:
        logs = []
        output_file = f"{log_type}_logs.csv"
        
        try:
            hand = win32evtlog.OpenEventLog(None, log_type)
            flags = win32evtlog.EVENTLOG_BACKWARDS_READ | win32evtlog.EVENTLOG_SEQUENTIAL_READ
            total_records = win32evtlog.GetNumberOfEventLogRecords(hand)
            
            while True:
                events = win32evtlog.ReadEventLog(hand, flags, 0)
                if not events:
                    break
                for event in events:
                    logs.append([event.TimeGenerated, event.EventID, event.SourceName, event.StringInserts])
        
        except Exception as e:
            print(f"Failed to read {log_type} logs: {e}")
            continue
        
        # Save logs to CSV
        with open(output_file, "w", newline='', encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["Timestamp", "EventID", "Source", "Details"])
            for log in logs:
                writer.writerow([log[0], log[1], log[2], log[3] if log[3] else "N/A"])
        
        print(f"Logs saved to {output_file}")

# Run the function to fetch logs from all specified types
fetch_windows_logs()



Logs saved to Application_logs.csv
Failed to read Security logs: (1314, 'OpenEventLogW', 'A required privilege is not held by the client.')
Logs saved to System_logs.csv
Logs saved to Setup_logs.csv
Logs saved to ForwardedEvents_logs.csv


In [1]:
import subprocess
import csv

def fetch_network_connections():
    output_file = "Network_logs.csv"

    # Run 'netstat' command to get active network connections
    result = subprocess.run(["netstat", "-ano"], capture_output=True, text=True)

    logs = []
    for line in result.stdout.split("\n"):
        parts = line.split()
        if len(parts) >= 5 and parts[0] in ["TCP", "UDP"]:
            logs.append(parts[:5])  # Protocol, Local Address, Foreign Address, State, PID

    # Save to CSV
    with open(output_file, "w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Protocol", "Local Address", "Foreign Address", "State", "PID"])
        writer.writerows(logs)

    print(f"Network logs saved to {output_file}")

fetch_network_connections()


Network logs saved to Network_logs.csv
